In [1]:
import os
# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
#! java -version

# Install pyspark
#! pip install --ignore-installed pyspark==2.4.5
! pip install --upgrade pyspark

# Install Spark NLP
! pip install --ignore-installed spark-nlp==2.4.5

# Install nltk
! pip install nltk

! pip install pyldavis

! pip install -q findspark

# Install geopandas
!pip install --upgrade geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 55 kB/s 
     |████████████████████████████████| 199 kB 71.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=f45fc0269b9ed82317f9abe48e1a20c441a46dbcd17f33db1b097d1539794c1e
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 110 kB 34.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 24.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
#import pyspark.sql.functions as F
from pyspark.sql.functions import * 
import sparknlp
from pyspark.sql.functions import udf
from sparknlp.annotator import Tokenizer
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Normalizer
from sparknlp.annotator import LemmatizerModel
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
eng_stopwords = stopwords.words('english')
from nltk.corpus import stopwords
from sparknlp.annotator import StopWordsCleaner
from sparknlp.annotator import NGramGenerator
from sparknlp.annotator import PerceptronModel
from sparknlp.base import Finisher
from pyspark.ml import Pipeline
from pyspark.sql import types as T
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
from pyspark.sql.types import *
import re
import geopy
from geopy.geocoders import Nominatim
from pyspark import StorageLevel
from pyspark.sql.functions import monotonically_increasing_id as mi

import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [4]:
!unzip /content/gdrive/Shareddrives/ukraine-russia/ukraine-russia.zip

Archive:  /content/gdrive/Shareddrives/ukraine-russia/ukraine-russia.zip
   creating: ukraine-russia/
  inflating: ukraine-russia/08.csv   
  inflating: __MACOSX/ukraine-russia/._08.csv  
  inflating: ukraine-russia/09.csv   
  inflating: __MACOSX/ukraine-russia/._09.csv  
  inflating: ukraine-russia/02.csv   
  inflating: __MACOSX/ukraine-russia/._02.csv  
  inflating: ukraine-russia/03.csv   
  inflating: __MACOSX/ukraine-russia/._03.csv  
  inflating: ukraine-russia/04.csv   
  inflating: __MACOSX/ukraine-russia/._04.csv  
  inflating: ukraine-russia/10.csv   
  inflating: __MACOSX/ukraine-russia/._10.csv  
  inflating: ukraine-russia/05.csv   
  inflating: __MACOSX/ukraine-russia/._05.csv  
  inflating: ukraine-russia/07.csv   
  inflating: __MACOSX/ukraine-russia/._07.csv  
  inflating: ukraine-russia/06.csv   
  inflating: __MACOSX/ukraine-russia/._06.csv  


In [20]:
spark = sparknlp.start()
df = spark.read.csv("./ukraine-russia/02.csv", header=True,  inferSchema=True)
df = df.filter(col("language")=="en")
#df.persist()
#df2.show(10)

In [21]:
df.show(10)

+----------+-------------------+---------------+--------------------+------------------+---------+---------+-----------+--------------------+-------------------+-------------------+------------+--------------------+--------------------+--------+-----------+--------------+--------------------+
|Unnamed: 0|             userid|       username|            acctdesc|          location|following|followers|totaltweets|       usercreatedts|            tweetid|     tweetcreatedts|retweetcount|                text|            hashtags|language|coordinates|favorite_count|         extractedts|
+----------+-------------------+---------------+--------------------+------------------+---------+---------+-----------+--------------------+-------------------+-------------------+------------+--------------------+--------------------+--------+-----------+--------------+--------------------+
|         6|         3318106141|      twilou200|                null|              null|     1439|      218|     14852

In [51]:
def preprocess_data(df):
  df = df.filter(df.text.isNotNull())
  #tweet_col = ('location','text')
  review_text = df.select("location","text").filter(col("location").isNotNull())
  clean = review_text.withColumn('text', regexp_replace('text', r"http\S+", ''))
  clean = clean.withColumn('text', regexp_replace('text', r"\w+followback\w+", ''))
  clean = clean.withColumn('text', regexp_replace('text', '#', ''))
  clean = clean.withColumn('text', regexp_replace('text', 'RT', ''))
  clean_text = clean.withColumn('text', regexp_replace('text', ':', ''))
  return clean_text

def getCountryName(tweetText):
  geolocator = Nominatim(timeout=10, user_agent = "geoapiExercises")
  location = geolocator.geocode(tweetText, language='en')    
  location = str(location)
  if location != "None":
    location = str(location).split(", ")
        #location = str(location)
    location = location[len(location)-1]
  else:
    location = "Null"
  
  return location

def getSentimentScore(tweetText):
  analyzer = SentimentIntensityAnalyzer()
  vs = analyzer.polarity_scores(tweetText)
  neg = vs['neg']
  neu = vs['neu']
  pos = vs['pos']
  comp = vs['compound']
     
  if neg > pos:
    strg = "Negative"
        #return neg, vs['neg']
  elif pos > neg:
    strg = "Positive"
        #return 2, vs['neu']
  else:
    strg = "Neutral"
        #return  vs['compound']
 
  return strg

def getNegSentiment(text):
  analyzer = SentimentIntensityAnalyzer()
  score = analyzer.polarity_scores(text)
  return score['neg']

def getPosSentiment(text):
  analyzer = SentimentIntensityAnalyzer()
  score = analyzer.polarity_scores(text)
  return score['pos']

def getNeuSentiment(text):
  analyzer = SentimentIntensityAnalyzer()
  score = analyzer.polarity_scores(text)
  return score['neu']

In [23]:
processed_tweets = preprocess_data(df)
print(processed_tweets.count())
processed_tweets.show(10,False)

44584
+------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|location          |text                                                                                                                                                                                                                                                   |
+------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Alabama, USA      |China is financing Russia’s aggression in Ukraine. Sanction China.                                                                                                     

In [52]:
#new = colLoc[0:3]
processed_tweetslim = processed_tweets.limit(10000).cache()
df1=open('./text file2','w')
processed_tweetslim2 = processed_tweetslim.select(processed_tweetslim["location"])

#convert DF to RDD
rDD_Tweet = processed_tweetslim2.rdd.map(list).persist()
locationrDD = rDD_Tweet.map(lambda x: getCountryName(x))
locationrDD.take(10)

#convert RDD back to DF
locDF = spark.createDataFrame(locationrDD, StringType())

#Join with original DF
id = mi()
locDF = locDF.withColumn("id1",id)
processed_tweetslim = processed_tweetslim.withColumn("id2",id)
joinnDF = locDF.join(processed_tweetslim,col("id1")==col("id2"),"inner").drop("id1","id2","location")
joinnDF = joinnDF.withColumnRenamed("value", "Country").filter(col("Country") != "Null").cache()
joinnDF.show(20)

+--------------+--------------------+
|       Country|                text|
+--------------+--------------------+
| United States|China is financin...|
| United States|GENEVA (AP) — Swi...|
|       Ukraine|🇺🇦🇧🇾 At the v...|
| United States|The UkraineInvasi...|
|       Namibia|Russia and Ukrain...|
|United Kingdom|❗️Switzerland wil...|
|        France|Out of desperatio...|
|      Slovenia|🚨🚨🇮🇱Urgent Is...|
| United States|r/Murderedbywords...|
| United States|@shivansh_880 @Ru...|
|United Kingdom|⚡️Moscow Exchange...|
|        Brazil|Schastia in E Ukr...|
|United Kingdom|Schastia in E Ukr...|
|        Canada|It’s day 5 of Rus...|
|       Germany|🇺🇦🇧🇾 At the v...|
|         India|Leaders of nation...|
| United States|Martin must have ...|
| United States|AFP The United_St...|
|United Kingdom|... those gippo's...|
|       Germany|"The GOP party wi...|
+--------------+--------------------+
only showing top 20 rows



In [53]:
joinnDF.count()

8569

In [ ]:
joinnDF.write.options(header='True', delimiter=',').csv("./geoLocations.csv")
df5.count()

In [57]:
udfNegSentimentScore = udf(getNegSentiment, FloatType())
udfPosSentimentScore = udf(getPosSentiment, FloatType())
udfNeuSentimentScore = udf(getNeuSentiment, FloatType())
udfSentimentScore = udf(getSentimentScore, StringType())

df3 = joinnDF.withColumn("posSentiment", udfPosSentimentScore("text"))\
.withColumn("negSentiment", udfNegSentimentScore("text"))\
.withColumn("neuSentiment", udfNeuSentimentScore("text"))\
.withColumn('Sentiment', udfSentimentScore('text')).cache()

df3.show(30)

+--------------+--------------------+------------+------------+------------+---------+
|       Country|                text|posSentiment|negSentiment|neuSentiment|Sentiment|
+--------------+--------------------+------------+------------+------------+---------+
| United States|China is financin...|         0.0|       0.216|       0.784| Negative|
| United States|GENEVA (AP) — Swi...|       0.264|       0.062|       0.674| Positive|
|       Ukraine|🇺🇦🇧🇾 At the v...|         0.0|         0.0|         1.0|  Neutral|
| United States|The UkraineInvasi...|       0.115|         0.0|       0.885| Positive|
|       Namibia|Russia and Ukrain...|         0.0|       0.166|       0.834| Negative|
|United Kingdom|❗️Switzerland wil...|         0.0|       0.106|       0.894| Negative|
|        France|Out of desperatio...|       0.052|       0.365|       0.583| Negative|
|      Slovenia|🚨🚨🇮🇱Urgent Is...|       0.115|         0.0|       0.885| Positive|
| United States|r/Murderedbywords...|         0.0| 

In [58]:
df3NEG = df3.select("Country","negSentiment","Sentiment").filter(col("Sentiment") == "Negative")
df3POS = df3.select("Country","posSentiment","Sentiment").filter(col("Sentiment") == "Positive")
df3NEU = df3.select("Country","neuSentiment","Sentiment").filter(col("Sentiment") == "Neutral")
df3POS.show(50)

+--------------+------------+---------+
|       Country|posSentiment|Sentiment|
+--------------+------------+---------+
| United States|       0.264| Positive|
| United States|       0.115| Positive|
|      Slovenia|       0.115| Positive|
|       Germany|       0.109| Positive|
| United States|       0.145| Positive|
| United States|       0.207| Positive|
|        Canada|       0.185| Positive|
| United States|       0.195| Positive|
| United States|       0.067| Positive|
|        Canada|       0.157| Positive|
|        Canada|       0.194| Positive|
|         India|       0.319| Positive|
| United States|        0.29| Positive|
|United Kingdom|       0.048| Positive|
|       Austria|       0.158| Positive|
| United States|       0.205| Positive|
|         India|       0.158| Positive|
| United States|        0.21| Positive|
|United Kingdom|       0.225| Positive|
| United States|       0.067| Positive|
|      Pakistan|       0.087| Positive|
|       Austria|       0.158| Positive|


In [59]:
xdf3NEG = df3NEG.groupBy("Country","Sentiment").mean("negSentiment")
xdf3NEG.sort(desc("avg(negSentiment)")).persist().show(100)
#xdf3NEG.show(100)

+--------------------+---------+-------------------+
|             Country|Sentiment|  avg(negSentiment)|
+--------------------+---------+-------------------+
|              Zambia| Negative| 0.6700000166893005|
|        Burkina Faso| Negative|0.37050001323223114|
|                Chad| Negative| 0.3490000069141388|
|             Algeria| Negative|0.34200000762939453|
|             Iceland| Negative|  0.335999995470047|
|               Syria| Negative|0.32600000500679016|
|            Eswatini| Negative| 0.3240000009536743|
|        Sierra Leone| Negative| 0.3149999976158142|
|              Warsaw| Negative|             0.3125|
|             Moldova| Negative| 0.3114999905228615|
|              Cyprus| Negative| 0.2884000062942505|
|Bosnia and Herzeg...| Negative| 0.2857499923557043|
|              Taiwan| Negative|0.27919999659061434|
|                Oman| Negative| 0.2760000005364418|
|             Tunisia| Negative|0.27000001072883606|
|               Libya| Negative| 0.26800000667

In [60]:
xdf3POS = df3POS.groupBy("Country","Sentiment").mean("posSentiment")
xdf3POS.sort(desc("avg(posSentiment)")).persist().show(20)

+--------------------+---------+-------------------+
|             Country|Sentiment|  avg(posSentiment)|
+--------------------+---------+-------------------+
|          Azerbaijan| Positive| 0.6039999723434448|
|               Syria| Positive| 0.5889999866485596|
|               Ghana| Positive|0.42499999701976776|
|            Paraguay| Positive|0.37700000405311584|
|       North America| Positive| 0.3400000035762787|
|          Mauritania| Positive|  0.335999995470047|
|              Zambia| Positive| 0.3199999928474426|
|              Cyprus| Positive| 0.3043333366513252|
|            Zimbabwe| Positive|0.28060000091791154|
|              Rwanda| Positive|0.27300000190734863|
|        Vatican City| Positive|0.26899999380111694|
|          Luxembourg| Positive|0.26899999380111694|
|                Laos| Positive|0.26750000193715096|
|United Arab Emirates| Positive| 0.2667500041425228|
|         El Salvador| Positive|0.26600000262260437|
|               Egypt| Positive|0.262666669984

In [61]:
xdf3NEU = df3NEU.groupBy("Country","Sentiment").mean("neuSentiment")
xdf3NEU.show(20)

+--------------------+---------+------------------+
|             Country|Sentiment| avg(neuSentiment)|
+--------------------+---------+------------------+
|              Monaco|  Neutral|               1.0|
|             Denmark|  Neutral|               1.0|
|            Djibouti|  Neutral|               1.0|
|United Arab Emirates|  Neutral|               1.0|
|             Ukraine|  Neutral|               1.0|
|         South Korea|  Neutral|               1.0|
|               Ghana|  Neutral|               1.0|
|              Canada|  Neutral|0.9945416667809089|
|             Finland|  Neutral|               1.0|
|        Saudi Arabia|  Neutral|               1.0|
|          Bangladesh|  Neutral|               1.0|
|             Belgium|  Neutral|               1.0|
|              Warsaw|  Neutral|               1.0|
|           Indonesia|  Neutral|               1.0|
|               Spain|  Neutral|0.9967000007629394|
|               Japan|  Neutral|0.9961538463830948|
|         El

In [62]:
pandas_dfNEG = xdf3NEG.toPandas()
pandas_dfPOS = xdf3POS.toPandas()
pandas_dfNEU = xdf3NEU.toPandas()
pandas_dfPOS

,Country,Sentiment,avg(posSentiment)
0,Liberia,Positive,0.121000
1,Tanzania,Positive,0.125000
2,South Korea,Positive,0.150000
3,Algeria,Positive,0.191000
4,Zambia,Positive,0.320000
...,...,...,...
120,Norway,Positive,0.157909
121,Switzerland,Positive,0.195882
122,Uruguay,Positive,0.145000
123,Iran,Positive,0.239286


In [240]:

fig = px.choropleth(pandas_dfPOS, locations="Country",
                    color_continuous_scale=px.colors.sequential.Plasma, title='Positive Sentiment',
                    locationmode= 'country names' , color='avg(posSentiment)',
                    range_color=[0,1],projection='natural earth',
                    scope ='world') 
fig.show()     

In [241]:
fig = px.choropleth(pandas_dfNEG, locations="Country",
                    color_continuous_scale=px.colors.sequential.Plasma, title='Negative Sentiment',
                    locationmode= 'country names' , color='avg(negSentiment)',
                    range_color=[0,1],projection='natural earth',
                    scope ='world') 
fig.show()    

In [242]:
fig = px.choropleth(pandas_dfNEU, locations="Country",
                    color_continuous_scale=px.colors.sequential.Plasma, title='Neutral Sentiment',
                    locationmode= 'country names' , color='avg(neuSentiment)',
                    range_color=[0,1],projection='natural earth',
                    scope ='world') 
fig.show()

In [18]:
fig = px.choropleth(pandas_dfPOS, locations="Country",
                    color_continuous_scale=px.colors.sequential.Plasma, title='Positive Sentiment month July',
                    locationmode= 'country names' , color='avg(posSentiment)',
                    range_color=[0,1],projection='natural earth',
                    scope ='world') 
fig.show()    

In [19]:
fig = px.choropleth(pandas_dfNEG, locations="Country",
                    color_continuous_scale=px.colors.sequential.Plasma, title='Negative Sentiment month July',
                    locationmode= 'country names' , color='avg(negSentiment)',
                    range_color=[0,1],projection='natural earth',
                    scope ='world') 
fig.show()

In [63]:
fig = px.choropleth(pandas_dfPOS, locations="Country",
                    color_continuous_scale=px.colors.sequential.Plasma, title='Positive Sentiment month February',
                    locationmode= 'country names' , color='avg(posSentiment)',
                    range_color=[0,1],projection='natural earth',
                    scope ='world') 
fig.show()    

In [64]:
fig = px.choropleth(pandas_dfNEG, locations="Country",
                    color_continuous_scale=px.colors.sequential.Plasma, title='Negative Sentiment month February',
                    locationmode= 'country names' , color='avg(negSentiment)',
                    range_color=[0,1],projection='natural earth',
                    scope ='world') 
fig.show()